In [1]:
#Import required liibraries
import IPython
import matplotlib.pyplot as plt
import IPython.display as Disp
from ipywidgets import widgets
import numpy as np
import cv2
import plotly.graph_objects as go
from ipywidgets import interact, interactive
import glob

%matplotlib widget

In [2]:
#Define a button 
startbutton = widgets.Button(description="Start Iterations")
output_box = widgets.Output()
display(output_box)

Output()

In [3]:
print('Tracking Dashboard')

Tracking Dashboard


In [4]:
#Define the mouse input parameters
class bbox_select():
    #%matplotlib notebook
    
    def __init__(self,im):
        self.tissues = 2
        self.im = im
        self.clicks = 0
        self.selected_points = []
        for i in range(self.tissues):
            self.selected_points.append([])
        self.fig,ax = plt.subplots()
        self.img = ax.imshow(self.im.copy())
        self.ka = self.fig.canvas.mpl_connect('button_press_event', self.onclick)
        disconnect_button = widgets.Button(description="Selected Posts")
        Disp.display(disconnect_button)
        disconnect_button.on_click(self.disconnect_mpl)

    def poly_img(self,img,pts):
        for point in pts:
            temp = np.array(point, np.int32)
            point = temp.reshape((-1,1,2))
            cv2.polylines(img,[point],True,(np.random.randint(0,255),np.random.randint(0,255),np.random.randint(0,255)),7)
            #cv2.rectangle(img, point[0], point[1])
        return img

    def onclick(self, event):
        for i in range(self.tissues):
            if len(self.selected_points[i])<2:   
                self.selected_points[i].append([event.xdata,event.ydata])
                break
                
        if len(self.selected_points[0])>1:
            self.fig
            self.img.set_data(self.poly_img(self.im.copy(),self.selected_points))

        
    def disconnect_mpl(self,_):
        self.fig.canvas.mpl_disconnect(self.ka)

In [5]:
class imageanalysis():
    def __init__(self):
        
        self.OPENCV_OBJECT_TRACKERS = {
            "csrt": cv2.TrackerCSRT_create,
            "kcf": cv2.TrackerKCF_create,
            "boosting": cv2.TrackerBoosting_create,
            "mil": cv2.TrackerMIL_create,
            "tld": cv2.TrackerTLD_create,
            "medianflow": cv2.TrackerMedianFlow_create,
            "mosse": cv2.TrackerMOSSE_create
        }
        
        self.trackers = cv2.MultiTracker_create()
        
        self.startbutton = widgets.Button(description="nothing")
        display(startbutton)
        startbutton.on_click(self.on_button_clicked)
    
    def setstream(self, path):
        self.videostream = cv2.VideoCapture(path)
        self.images = self.videostream.read()[1]
        self.bs = bbox_select(self.images)
        #self.modifybox(self.bs.selected_points)
        
    def modifybox(self, badpoints):
        boxes = badpoints

        for i in range(len(badpoints)):
            count = 0
            for (xpoint, ypoint) in badpoints[i]:
                if (count % 2) == 1:
                    boxes[i] = (xprev, yprev, xpoint - xprev, ypoint - yprev)
                else:
                    xprev = xpoint
                    yprev = ypoint
                count+=1
        return boxes

    def on_button_clicked(self, b):
        box = self.modifybox(self.bs.selected_points)

        for i in range(len(box)):
            tracker = self.OPENCV_OBJECT_TRACKERS['csrt']()
            self.trackers.add(tracker, self.images, box[i])

        count = 0
        lists = []
        xox = []

        fig = go.Figure()
        trace = fig.add_trace(go.Scatter(x=xox,y=lists)) 
        while True:
            #count = count + 1
            print(count)
            if count >= 100:
                break
            self.image = self.videostream.read()[1]
            if self.image is None:
                break
            posts = self.trackers.update(self.image)[1]
            postcords = []

            for post in posts:
                print('yeye')
                #Used float for more acuracy but rectangle needs int
                (x, y, w, h) = [float(i) for i in post]
                (rx, ry, rw, rh) = [int(i) for i in (x, y, w, h)]
                cv2.rectangle(self.image, (rx, ry), (rx + rw, ry + rh), (0, 255, 0), 2)
                #Populate list for centroid tracking
                postcords.append((x, y, x + w, y + h))  


            postio = {}

            for (i, (x, y, x2, y2)) in enumerate(postcords):
                    print('here')
                    centroidX = int((x + x2) / 2.0)
                    centroidY = int((y + y2) / 2.0)
                    postio[i] = (centroidX, centroidY)
            #print(postio)

            for (objectID, centroid) in postio.items():
                #print(centroid)
                #print('there')
                #text = "{}".format(objectID)

                #cv2.putText(image, text, (centroid[0] - 10, centroid[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                #cv2.circle(image, (centroid[0], centroid[1]), 4, (0, 255, 0), -1)
                #plt.imshow(image)

                if (objectID % 2) == 0:
                        #Save the x position of the even post
                        evenX = centroid[0]
                        #Save the y position of the even post
                        evenY = centroid[1]
                        evenID = objectID
                        #If objectID is odd it has a pair so do stuff
                elif (objectID - 1) == evenID:
                        #Calculate tissue number based on object ID
                        reltissueID = int((objectID - 1)/2)
                        #Save the x position of the odd post
                        oddX = centroid[0]
                        #Save the y position of the odd post
                        oddY = centroid[1]

                        disp = np.sqrt(((oddX - evenX)**2) + ((oddY - evenY)**2))
                        xox.append(count)
                        count = count + 1
                        print(count)
                        lists.append(disp)
                trace.data[0].x = xox
                trace.data[0].y = lists
            #trace = fig.add_trace(go.Scatter(x=xox,y=lists)) 
                with output_box:
                    output_box.clear_output(wait=True)
                    display(fig)#= fig.add_trace(go.Scatter(x=xox,y=lists)) 

        return box
analyze = imageanalysis()



Button(description='Start Iterations', style=ButtonStyle())

0
yeye
yeye
here
here
1
1
yeye
yeye
here
here
2
2
yeye
yeye
here
here
3
3
yeye
yeye
here
here
4
4
yeye
yeye
here
here
5
5
yeye
yeye
here
here
6
6
yeye
yeye
here
here
7
7
yeye
yeye
here
here
8
8
yeye
yeye
here
here
9
9
yeye
yeye
here
here
10
10
yeye
yeye
here
here
11
11
yeye
yeye
here
here
12
12
yeye
yeye
here
here
13
13
yeye
yeye
here
here
14
14
yeye
yeye
here
here
15
15
yeye
yeye
here
here
16
16
yeye
yeye
here
here
17
17
yeye
yeye
here
here
18
18
yeye
yeye
here
here
19
19
yeye
yeye
here
here
20
20
yeye
yeye
here
here
21
21
yeye
yeye
here
here
22
22
yeye
yeye
here
here
23
23
yeye
yeye
here
here
24
24
yeye
yeye
here
here
25
25
yeye
yeye
here
here
26
26
yeye
yeye
here
here
27
27
yeye
yeye
here
here
28
28
yeye
yeye
here
here
29
29
yeye
yeye
here
here
30
30
yeye
yeye
here
here
31
31
yeye
yeye
here
here
32
32
yeye
yeye
here
here
33
33
yeye
yeye
here
here
34
34
yeye
yeye
here
here
35
35
yeye
yeye
here
here
36
36
yeye
yeye
here
here
37
37
yeye
yeye
here
here
38
38
yeye
yeye
here
here
39
39
ye

In [6]:
class fileselection():
    def __init__(self):
        self.folders = glob.glob('Videos/*')
        self.filepath = None
        
        for i in range(len(self.folders)):
            self.folders[i] = self.folders[i].split('/')[1]

        display(interactive(self.loadfiles, Folders=self.folders))

    def choosevideo(self, Videos):
        self.filepath = Videos
        analyze.setstream(Videos)

    def loadfiles(self, Folders):
        files = sorted(glob.glob('Videos/' + Folders + '/*.mp4'))
        display(interactive(self.choosevideo, Videos=files))
vid = fileselection()

interactive(children=(Dropdown(description='Folders', options=('080819',), value='080819'), Output()), _dom_cl…

In [22]:
print(vid.filepath)

Videos/080819/video-Copy2.mp4


In [ ]:

#box = modifybox(bs.selected_points)

In [ ]:
out = widgets.Output()

@out.capture(clear_output=True, wait=True)
def function_with_captured_output():
    with out:
        print('This goes into the output widget')
    with out:
        print('other')
    

function_with_captured_output()
out